# DyNet RNNs チュートリアル

チュートリアルを意訳しています。

参考
* https://github.com/clab/dynet/blob/master/examples/python/tutorials/RNNs.ipynb
* https://media.readthedocs.org/pdf/dynet/latest/dynet.pdf

In [1]:
from dynet import *

# LSTM/RNN 全体像

１層のRNNは隠れ層を次のように分けている $h_1,...,h_k$, $h_i$ は時間ごとの指標を表している
それぞれの層$h_i$は入力$x_i$と出力$r_i$に加えて過去の入力$r_{i-1}$も与える
複数層の場合は連続ではなく網目状になる。例えば

* $h_1^3,...,h_k^3$
* $h_1^2,...,h_k^2$
* $h_1^1,...h_k^1$

上記の場合は

隠れ層$h_i^1$の入力は$x_i$と$r_{i-1}^1$

隠れ層$h_i^2$の入力は$r_i^1$と$r_{i-1}^2$　など

# LSTM インターフェース

RNN / LSTM / GRU は同様のインターフェースになる。
"builder"が連続列のためのパラメータを作成する

In [2]:
model = Model()
NUM_LAYERS = 2
INPUT_DIM = 50
HIDDEN_DIM = 10
builder = LSTMBuilder(NUM_LAYERS, INPUT_DIM, HIDDEN_DIM, model)

"Builder"でRNNの内部パラメータをモデルに加えることでパラメータを意識しなくて良くなるが最適化の際は同時に行ってくれる！！

In [3]:
s0 = builder.initial_state()
s0

In [4]:
print(vecInput(10))
x1 = vecInput(INPUT_DIM)
print(x1)

expression 22/0
expression 23/0


In [5]:
s1 = s0.add_input(x1)
y1 = s1.output()

In [6]:
y1.npvalue().shape

(10,)

In [7]:
s1.h()

(expression 33/0, expression 43/0)

In [8]:
s2 = s1.add_input(x1)
y2 = s2.output()
y2

expression 67/0

１層の層で$y2$は隠れ層の状態と等しい。しかしながら最終層でもある。
隠れ層の状態にアクセスしたい場合は下記のメソッドを使用する

In [9]:
s2.h()

(expression 55/0, expression 67/0)

LSTMの状態の出力を比較してみると

In [10]:
print("all layers:", s1.h())

all layers: (expression 33/0, expression 43/0)


In [11]:
print( s1.s())

(expression 29/0, expression 39/0, expression 33/0, expression 43/0)


同様のインターフェースでシンプルなRNNも作成可能

In [12]:
rnnbuilder = SimpleRNNBuilder(NUM_LAYERS, INPUT_DIM, HIDDEN_DIM, model)

rs0 = rnnbuilder.initial_state()

rs1 = rs0.add_input(x1)
ry1 = rs1.output()
print( rs1.s())

(expression 75/0, expression 77/0)


* 出力は"output"メソッドで行う
* 隠れ層の値は"h()"メソッドで行う
* 状態は"s()"で確認する

LSTMのみ出力が４つになっている。これは入力層、忘却層、過去の入力層、出力層が存在するためである。

In [13]:
rnn_h = rs1.h()
rnn_s = rs1.s()
print("RNN h:", rnn_h)
print("RNN s:", rnn_s)

lstm_h = s1.h()
lstm_s = s1.s()
print("LSTM h:", lstm_h)
print("LSTM s:", lstm_s)

RNN h: (expression 75/0, expression 77/0)
RNN s: (expression 75/0, expression 77/0)
LSTM h: (expression 33/0, expression 43/0)
LSTM s: (expression 29/0, expression 39/0, expression 33/0, expression 43/0)


# RNNとLSTMインターフェースのオプション

Stack LSTM: 過去の状態を覚えておけるのでそこに新たに追加してグラフの作成が行える。



In [14]:
s2 = s1.add_input(x1)
s3 = s2.add_input(x1)
s4 = s3.add_input(x1)

# s3に新しい入力を加える
s5 = s3.add_input(x1)

#下記の２つの異なる列ができている
# s0, s1, s2, s3, s4
# s0, s1, s2, s3, s5

assert(s5.prev() == s3)
assert(s4.prev() == s3)

s6 = s3.prev().add_input(x1)
# s0, s1, s2, s6

In [15]:
s6.h()

(expression 185/0, expression 197/0)

4つの列が出力される

In [16]:
s6.s()

(expression 181/0, expression 193/0, expression 185/0, expression 197/0)

# メモリの効果的な変換

入力列をlist化して今までのことを一気にやってくれる手法があります。

In [17]:
state = rnnbuilder.initial_state()
xs = [x1, x1, x1]
states = state.add_inputs(xs)
outputs = [s.output() for s in states]
hs = [s.h() for s in states]
print(outputs)
print(hs)

[expression 201/0, expression 205/0, expression 209/0]
[(expression 199/0, expression 201/0), (expression 203/0, expression 205/0), (expression 207/0, expression 209/0)]


transduceを使用すればもっとメモリを効率よく使用でき、少ない行数でかけます。

In [18]:
state = rnnbuilder.initial_state()
xs = [x1, x1, x1]
outputs = state.transduce(xs)
print(outputs)

[expression 213/0, expression 217/0, expression 221/0]


# 言葉レベルのLSTM

ここから本格的な応用に入ります。言葉レベルのRNNですprint(outputs)

* モデルのパラメータの初期的設定
* 言葉の設定。
* 明示的に終了と始まり符号を表す<EOS>を追加
* コンピューターで扱うため、数字と文字の変換用の辞書を用意

In [19]:
import random
from collections import defaultdict
from itertools import count
import sys

LAYERS = 2
INPUT_DIM = 50 
HIDDEN_DIM = 50  

characters = list("ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz ,.")
characters.append("<EOS>")

int2char = list(characters)
char2int = {c:i for i,c in enumerate(characters)}

VOCAB_SIZE = len(characters)

# モデルの設定

* modelのパラメータ設定
* 61ページにlookupパラメータの意味があります。Word Embeddingのためです。
http://phontron.com/slides/emnlp2016-dynet-tutorial-part1.pdf
* params["lookup"]は入力用
* params["R"]は出力用
* params["bias"]はバイアス用

In [20]:
model = Model()


srnn = SimpleRNNBuilder(LAYERS, INPUT_DIM, HIDDEN_DIM, model)
lstm = LSTMBuilder(LAYERS, INPUT_DIM, HIDDEN_DIM, model)

params = {}
params["lookup"] = model.add_lookup_parameters((VOCAB_SIZE, INPUT_DIM))
params["R"] = model.add_parameters((VOCAB_SIZE, HIDDEN_DIM))
params["bias"] = model.add_parameters((VOCAB_SIZE))

* 一つの文章に対してのメソッド：文章ごと長さが異なり処理を分けたいため定義
* pick関数は資料の38ページ参照
https://media.readthedocs.org/pdf/dynet/latest/dynet.pdf

In [21]:
# return compute loss of RNN for one sentence
def do_one_sentence(rnn, sentence):
    # setup the sentence
    renew_cg()
    s0 = rnn.initial_state()
    
    
    R = parameter(params["R"])
    bias = parameter(params["bias"])
    lookup = params["lookup"]
    sentence = ["<EOS>"] + list(sentence) + ["<EOS>"]
    sentence = [char2int[c] for c in sentence]
    s = s0
    loss = []
    for char,next_char in zip(sentence,sentence[1:]):
        s = s.add_input(lookup[char])
        probs = softmax(R*s.output() + bias)
        loss.append( -log(pick(probs,next_char)) )
    loss = esum(loss)
    return loss

モデルからの文字列生成

In [22]:
# generate from model:
def generate(rnn):
    def sample(probs):
        rnd = random.random()
        for i,p in enumerate(probs):
            rnd -= p
            if rnd <= 0: break
        return i
    
    # setup the sentence
    renew_cg()
    s0 = rnn.initial_state()
    
    R = parameter(params["R"])
    bias = parameter(params["bias"])
    lookup = params["lookup"]
    
    s = s0.add_input(lookup[char2int["<EOS>"]])
    out=[]
    while True:
        probs = softmax(R*s.output() + bias)
        probs = probs.vec_value()
        next_char = sample(probs)
        out.append(int2char[next_char])
        if out[-1] == "<EOS>": break
        s = s.add_input(lookup[next_char])
    return "".join(out[:-1]) # strip the <EOS>

学習のための関数

In [23]:
def train(rnn, sentence):
    trainer = SimpleSGDTrainer(model)
    for i in range(200):
        loss = do_one_sentence(rnn, sentence)
        loss_value = loss.value()
        loss.backward()
        trainer.update()
        if i % 5 == 0:
            print(loss_value)
            print(generate(rnn))

#実行例 RNN

In [24]:
sentence = "a quick brown fox jumped over the lazy dog"
train(srnn, sentence)

176.98768615722656
.zmwVrL Pgmgez
118.57496643066406
KWXCoVasCm
71.47085571289062
GZ l.,ee orew.UvtcsoxdoSBoPbcHi quvcR Vzn UWfkGxoo, tec l or  u NT perYahn dWJ erawTziNupcr Kqompruemko ghfowE KMoSoaeox ,Tokep
42.194435119628906
n xOod dqoh ddoJJr mfeI ouer ZYe d,oW m VzG Rkm l JwW qXo rp e brsnk ohex txe  aet jlg dZogzoXtr tvfyQtfC uod jSqcdPYjCuaseddozTG mrkrY  hoW tdmsxwdozdlueP eRhp.  CzO d uUer GrgPxxuyg odloXeG d.n trSacVC oxfn qIuYvaAjcm w  gxi  pdixjurQu iz uwoodjpdoyJK xgoHdi l,Zjw uurx qQar ovM TZnH   WZe  rutUt pavS beo
19.207962036132812
G Puc
6.561734199523926
J quick brown fox jumped ovew qSe lazyYdoc
2.2133874893188477
a quick brown fox jumped over the laze dog
0.8910670876502991
a quick brown fox jumped over the paay dog
0.6027364730834961
a quiik brown fox jumped over the lazy dog
0.4551372230052948
a quick brown fox jumped over ahe lazy dog
0.3652839660644531


a quick brown fox jumped over the lazy dog
0.3048505485057831
a quick brown fox jumped over the lazy dog
0.2614337205886841
a quick brown fox jumped over the lazy dog
0.22875484824180603
a quick brown fox jumped over the lazy dog
0.2032642662525177
a quick brown fox jumped over the lazy dog
0.18284228444099426
a quick brown fox jumped over the lazy dqg
0.16611282527446747
a quick brown fox jumped over the lazy dog
0.15216544270515442
a quick brown fox jumped over the lazy dog
0.14035697281360626
a quick brown fox jumped over the lazy dog
0.13023392856121063
a quick brown fox jumped over the lazy dog
0.12145433574914932
a quick brown fox jumped over the lazy dog
0.1137743815779686


a quick brown fox jumped over the lazy dog
0.10699939727783203
a quick brown fox jumped over the lazy dog
0.10097689926624298
a quick brown fox jumped over the lazy dog
0.09559248387813568
a quick brown fox jumped over fhe lazy dog
0.09074804931879044
a quick brown fox jumped over the lazy dog
0.08636464923620224
a quick brown fox jumped over the lazy dog
0.0823831781744957
a quick brown fox jumped over the lazy dog
0.0787506252527237
a quick brown fox jumped over the lazy dog
0.07541273534297943
a quick brown fox jumped over the lazy dog
0.07235343754291534
a quick brown fox jumped ouer the lazy dog
0.06952950358390808
a quick brown fox jumped over the lazy dog
0.06690983474254608


a quick brown fox jumped over the lazy dog
0.06448080390691757
a quick brown fox jumped over the lazy dog
0.06222354620695114
a quick brown fox jumped over the lazy dog
0.060115743428468704
a quick brown fox jumped over the lazy dog
0.05814079940319061
a quick brown fox jumped over the lazy dog
0.05629827454686165
a quick brown fox jumped over the lazy dog
0.054562509059906006
a quick brown fox jumped over the lazy dog
0.052930597215890884
a quick brown fox jumped over the lazy dog


#実行例 LSTM

In [25]:
sentence = "a quick brown fox jumped over the lazy dog"
train(lstm, sentence)

163.1386260986328
.nfWanMWoocnpxkbPCZ igPmOrnBWBHz bFR xe.DLzXG mqcYIdf eieMWH xoLeeadOhXCePUKtxH
141.42947387695312
pf rL   bqP koe r ,jejes  ogreshOXf oaNo  z rjohx   e,bo  oTzgd  v 
134.0388946533203
EPohmzejds drhfoUJltuuocoh Colugox oukokywo 
128.60665893554688
n 
116.18101501464844
je
101.5512466430664


ua dxuBWddaie Ztme  Hoj
86.95086669921875
Irqeolrif oaw jdoonCo dooxw tkGddc
75.145263671875
N dSLrelO ubmap egumeebdoo
63.167720794677734
BAr oez qhe lyo lxpdlyody ouox mhh lhn fWfhlttf lap tie tua w Gpe fhp rrufy  obe ja
51.71269226074219


a iZr roxx ozrd ogg y foze ox
43.78972244262695
z quiic obrwn lgyp dovand dogrnjthe olpnedqvre dhopa ffv
32.906009674072266
p lGc doo
26.622600555419922
c  rir drox jumpp.F go
19.275283813476562
n doo
13.499739646911621


a quikr eatn foz jumpped ovea lmzy dogz junped over the lazzy bogznd oxqn hrlwa ngo nd ov
9.051881790161133

6.094839572906494
U ouick brownndoxvn tox
3.9135212898254395
z quukk brown fox jumped over the lazy dog
2.145503044128418
a quick brown fox jumped overt th fox jumped over the lazy dog
1.5657320022583008


a quick brown fox jumped oven the lazy dog
1.2290349006652832
a quik brown fox jumped over the lazy dog
1.0082154273986816
a quick brown fox jumped over the lazy dog
0.8528245091438293
a quick brown fog jumped over the lazy dog
0.7378641366958618
a quick brown fox jumped over the lazy dog
0.6495485901832581


a quick brown fox jumped over the lazy dog
0.5796661972999573
a quick brown fox jumped over the lazy dog
0.5230434536933899
a quick brown fox jumped over the lazy dog
0.47626322507858276
a quick brown fox jumped over the lazy dog
0.4369813799858093
a quick brown fox jumped over the lazy dog


0.4035414755344391
a quick brown fox jumped over the lazy dog
0.3747418522834778
a quick brown fox jumped over the lazy dog
0.34968844056129456
a quick brown fox jumped over the lamy dog
0.3276979327201843
a quick brown fox jumped over the lazy dog
0.3082388937473297


a quick browr the lazy dog
0.2909145951271057
a quick browz fox jumped over the lazy dog
0.275386780500412
a quick brown fox jumped over the lazy dog
0.26138970255851746
a quick brown fox juzped over the lozy dog
0.24871425330638885
a quick brown fox jumped over the lazy dog
0.23718073964118958


a quick brown fox jumped over the lazy dog
0.2266462743282318
a quick grown fox jumped over the lazy dog


In [36]:
sentence = "I have a pen. I have an apple. Apple pen. " \
           "I have a pen. I have a Pineapple. Pineapple pan." \
           "Apple pen, Pineapple Pen, Pineapple Apple Pen " \
           "PPAP PPAP PPAP"
train(lstm, sentence)

581.0768432617188
a quick brown fox jumped over mhe lazy dog
80.7076187133789
a pen b
29.160541534423828


a q qan.f bpee. Pineapple PenP PxPe
20.348590850830078
a Pineapple pan
14.379410743713379


I have a pen. I have a pen. I have a pen. I have a pen. I have an apple. Apple pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a Pineapple. Pineapple pan.j Pineapple pan.Apple pen. I have a pen. I have a pen. I have a pen. I have an apple. Apple pen. I have a pen. I have a pen. I have an apple. Apple a Pineapple. Pineapple pan fPAP
11.341286659240723
I have a pen. I have a pen. I have a PineapplemAP
12.960704803466797


I have a
11.388411521911621
I have a pen. I have a pen. I Lave a Pineapple. Paneapple pan.Apple pen, Pineapple Pen, Pineapple Pen, Pineapple pan.Apple pen, Pineapple Apple Pen PPAP PPAP PPAP PPAP PPAA PPAP PPAP PPAP PPAP PPAP PPAP PPAP PPAP
11.969993591308594


I have a PiAP
10.483719825744629
I have an apple. Apple pen. I have an apple. Apple pen. I have a pen. I have a Peneapple. Pineapple pan.Apple pen, Pineapple pan.Apple.p PPAP
18.223918914794922

...


I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.9161453247070312


I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.869955062866211
I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.8431320190429688


I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.8111820220947266
I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.7916518449783325


I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.767293930053711


I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.751880407333374
I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP

In [26]:
sentence = "I have a pen. I have an apple. Apple pen. " \
           "I have a pen. I have a Pineapple. Pineapple pan." \
           "Apple pen, Pineapple Pen, Pineapple Apple Pen " \
           "PPAP PPAP PPAP"
train(lstm, sentence)

1148.27490234375
a quick brown fox jumped over the lazy dog
642.880615234375
a quick brown fe azp xod


475.9980163574219
a uumck cowpplpglapeoalk overpznkpco
391.21875


a dbog icrlPne le ape lne lnlnzp h, pppe Pg Pnl e . jncn n Pah aype aPnlpya e nera nae P PaPp olp Pox be Pnp l H tenpe l x gne ape la y c hnl ape azmb l
333.8431701660156
a uuccaa gleenOne Peapllyelappegl PKrep.ppex fePwe
311.44903564453125


a dog
259.00103759765625
a quick kognnpAlppP.vPip.pe.ppeAppk lAr hPPpe hP.plPe Pn apePPAppPlPpPk P.eppnAapk brPape vepl.pppPPe cenApPpeonnpA, phplie
242.81155395507812


a uurpla
207.337890625
a quipk halpppe. PeAPeeappove PnperpAplen.. yn.enpe bePpeApped.penpePlp.le Ph.ppek epPpe apenle wpePpppeeApplen. Niieneer AapelpePppePpe . apee. pn.Gpoen. ppeAApovenppeepPpe.ve Penk PeApppe PerPe penpppe PMAPyeeA enle l pelpevpeaI mped .p

...


a pen. I have azplle. PPAP PPAP PPAP
21.505712509155273
I have an apple. Apple pen. I have a pen. II.ave a pen. I have a Pineapple pan.Applelpple pen. Pineapple Pen, Pineapple Apple pen. A Pineapple pen, Pineapple pan.Apple Pen PPAP PPAP PPAP PPAPe Pen PPAP
18.25023651123047


I have a pen. I have a Pineapple. Pineapple Pen. PPAP
17.13021469116211
I have an Apple. Pineapple Apple Pen, Pineapple pan.Apple pen. I have a pen. I have a pen. I have a pen. I have an apple. Apple peAplle. Pineapple. Pineapple Pen, Pineapple pan.Apple Pen PPAP PPAP PPAP PPAP PPAP PPAP PAP
16.501943588256836


I have a Pineapple. Pineapple Pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP
15.226860046386719
ve an apple. Apple pen. I have a pen. I have an apple. I have an apple. Apple pen. I have a pen. I have a Pin.apple. Apna. Pineapple Apple Pen PPApple pen, Pineapple Pen, Pineapple pan.Apple Pen PPAP PPPAp PPen PPAP PPAP
13.21268081665039


I have a Pineapple. Pineapple pan.n, Pineapple Pen, Pineapple Pen, Pineapple Pen, Pineapple Pen, Pineapple Pen, PPineapple Pen, Pineapple pan.Apple pen, Pineapple Pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP P,PP
16.582279205322266
I have a pen. I have a Pineapple. Pineapple Pen, Pineapple Apple Pen PPAP PPAP
12.092254638671875


I have a Pineapplev. Apple pen. I have a Pineappleqpen. Pen I have an apple. Apple pen. I have an apple pan.apple. Pineapple Pen, Pineapple pan.e, Pineapple Pen, Pineapple Apple Pen PPAPppPAP PPAP PPAi PPAP PPAP PPAP PPAP PPAP PPAP PPAP PPAP PPAP PPPPA
12.637045860290527
I have a pen, Pineapple pan.Apple pen, PineApple pan.Apple pen, PiAn Apple penpe a, Pineapple pan.Apple pen. I have a pen. I have a pen. I have a pen. I have a pineapple. Apple pen. I have a pen. I have a pen. I have a Pineapple. Pineapple pen. I have an apple. Apple pen. I have a pen. I have an apple.. I have a pen. I have a pen. I have a pen. I have a pen. I have an apple. Apple pen. I have an apple. Apple pen. I have a pen. I have avple pen. I have a Pineapple. Pineapple pan.e, Pineapple pan.Apple pen, Pineapple pan.Apple pen, Pineapple Apple Pen PPAP PPAP

In [27]:
sentence = "I have a pen. I have an apple. Apple pen. " \
           "I have a pen. I have a Pineapple. Pineapple pan." \
           "Apple pen, Pineapple Pen, Pineapple Apple Pen " \
           "PPAP PPAP PPAP"
train(lstm, sentence)

12.392200469970703
k have a pen. I have an apple. Pineapple pan.n. Pineapple. Pineapple pan.Apple pen, Pineapple pan.Apple pen, Pineapple Pen, Pineapple Pen, Pineapple Apple Pen PPAP Pineapple Apple Pen PPAP PPn, PPn, PPnAp la Pin apple pan.Apple
11.039009094238281


I have a pen. I have an apple. Apple pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have a pen. I have an Apple pen. I have a pen. I have a Pineapple. Pineapple pen. I have an apple. Apple pen. I have a Pineapple. Pineapple pan.Apple pen. I have a pen. I have a pen. I have a pen. I have an apple. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple pan.Apple pen, Pineapple pan.Apple pen, PineApple pan.Apple pen, Pineapple Pen, Pineapple pan.APple pen, Pineapple pan.Apple pen, Pineapple pan.Apple pen, Pineapple Pen, Pineapple Pen, Pineapple pan,APple pen, PAP
11.342870712280273


I have a pen. I have a Pineapple. Pineapple pan.Apple pen, PPPAP Pineapple Apple Pen PPAP PPAP
10.583536148071289
I have a pen. I have a Pineapple. Pineapple , Pineapple Pen, Pineapple Apple Pen PPAP PPrPPPPAPPP APPPAP
9.70651912689209


n apple. Apple pen. I have a pen. I have a pen. I have a pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple pan.Apple pen, Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
11.42435073852539
I have a pen. I have an apple. ven, Pineapple Apple Pen PPAP PPAP PPAP
10.252484321594238


I have an apple. Apple pen. I have a pen. I have a pen. I have a Pineapple. Pineapple pan.n. I have a pen. I have a pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Apple Pen PPAp PPAP PPAP
9.39427375793457
I have an apple. Apple pen. I have an apple. Apple pen. I have a Pineapple. Pineapple Apple Pen PPAP PPAP PPAP
9.088889122009277

...


I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, PineapAle Apple Pen PPAP PPAP PPAP
1.9663851261138916
I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP


1.8968250751495361
I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.8381050825119019


I have a pen. I have an apple. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.7939939498901367
I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP


1.7554216384887695
I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.7247018814086914


I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP


In [28]:
sentence = "I have a pen. I have an apple. Apple pen. " \
           "I have a pen. I have a Pineapple. Pineapple pan." \
           "Apple pen, Pineapple Pen, Pineapple Apple Pen " \
           "PPAP PPAP PPAP"
train(lstm, sentence)

1.6977767944335938
I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.675619125366211


I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.6560249328613281
I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.6394424438476562


I have a pen. I have an apple. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.6246063709259033
I have a pen. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.611776351928711


I have a pen. I have an apple. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.6001302003860474
I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP


1.589889645576477
I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP

...


I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.4775687456130981
I have a pen. I have an apple. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP


1.4758434295654297
I have a pen. I have an apple. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.474153757095337


I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.4725505113601685
I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP


1.470999836921692
I have a pen. I have an apple. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.469527244567871


I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP


In [28]:
sentence = "I have a pen. I have an apple. Apple pen. " \
           "I have a pen. I have a Pineapple. Pineapple pan." \
           "Apple pen, Pineapple Pen, Pineapple Apple Pen " \
           "PPAP PPAP PPAP"
train(lstm, sentence)

1.6977767944335938
I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.675619125366211


I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.6560249328613281
I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.6394424438476562


I have a pen. I have an apple. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.6246063709259033
I have a pen. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.611776351928711


I have a pen. I have an apple. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.6001302003860474
I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP


1.589889645576477
I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP

...


I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.4775687456130981
I have a pen. I have an apple. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP


1.4758434295654297
I have a pen. I have an apple. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.474153757095337


I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have an apple. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.4725505113601685
I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP


1.470999836921692
I have a pen. I have an apple. Apple pen. I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
1.469527244567871


I have a pen. I have a Pineapple. Pineapple pan.Apple pen, Pineapple Pen, Pineapple Apple Pen PPAP PPAP PPAP
